In [17]:
from zhipuai import ZhipuAI
import json
import mysql.connector

# 读取配置文件
confige_path = "D:/code/python_code/ai/zhipu/confige.json"
with open(confige_path,"r",encoding="utf-8") as f:
    confige = json.load(f)  

sourcefile_path = confige["sourcefile_path"]
aitrans_path = confige["aitrans_path"]
filter_path = confige["filter_path"]
filtered_output_path = confige["filtered_output_path"]
transjson_path = confige["transjson_path"]


通过智谱ai的api将源数据转换成（题目-选项-答案-解析）格式

In [ ]:
client = ZhipuAI(api_key="297dc7f0e8dcbe31a21426081d8e9ee0.7OlrOM13fTTnhWdP")

"""
    完成情况:common1.txt文件已经完成转换
"""
#获取文件行数，用于循环
line_count = 0
with open(sourcefile_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    line_count = len(lines)

for line in range(1, line_count):
    data = ""
    with open(sourcefile_path, "r", encoding="utf-8") as f:
        data = lines[line-1]
    messages = [
{"role":"user",
 "content":"我的要求：不要输出我说的话；格式:(题目\n选项:A. \nB. \nC. \nD. \n答案\n解析:\n);每一个选项里都要有内容，解析只要一行，我提供的信息:"+data} ]
    response = client.chat.completions.create(messages= messages,model="glm-4-flash")
    with open(aitrans_path, "a", encoding="utf-8") as f:
        f.write(response.choices[0].message.content+"\n")


对转换后的数据进行数据处理

In [ ]:
"""
    获取没有空行的数据文件
"""
# 读取文件并过滤空行
with open(aitrans_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    filtered_lines = filter(lambda x: x.strip(), lines)
# 将过滤后的行写入新的文件
with open(filter_path, "w", encoding="utf-8") as f:
    f.write("".join(filtered_lines))

"""
    过滤文件中损坏的数据
"""
line_number = 0
lines_to_remove = []
# 读取文件并确定要删除的行
with open(filter_path, 'r', encoding='utf-8') as file:
    for line in file:
        line_number += 1
        if line.strip() == "题目":
            lines_to_remove.append(line_number)
            flag = True
        if line.startswith("题目") and len(line) > len("题目："):
            flag = False
        if flag:
            lines_to_remove.append(line_number)
# 写入文件时排除要删除的行
with open(filter_path, 'r', encoding='utf-8') as file, \
     open(filtered_output_path, 'w', encoding='utf-8') as filtered_file:
    current_line_number = 0
    for line in file:
        current_line_number += 1
        if current_line_number not in lines_to_remove:
            filtered_file.write(line)

"""
    检查数据中是否还有损坏的数据
"""
line_number = 0
with open(filtered_output_path, 'r', encoding='utf-8') as file:
    for line in file:
        line_number += 1
        if line.strip() == "题目":
            print(line_number)



将文件格式转换成json格式

In [22]:
def convert_to_json(file_path):
    # 初始化一个空列表来存储数据
    data = []

    # 打开文件并逐行读取
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # 初始化一个字典来存储当前问题的数据
    current_question = {}

    # 遍历每一行并处理它们
    for i, line in enumerate(lines):
        line = line.strip()  # 移除行首行尾的空白字符

        # 根据行的位置确定其角色
        if i % 8 == 0:
            current_question['题目'] = line[3:] if line.startswith('题目：') else line
        elif i % 8 == 1:
            current_question['选项'] = {}
        elif i % 8 in [2, 3, 4, 5]:
            option = line[0]  # 提取选项字母（A, B, C, 或 D）
            current_question['选项'][option] = line[2:]  # 存储选项的内容
        elif i % 8 == 6:
            current_question['答案'] = line[3:] if line.startswith('答案：') else line
        elif i % 8 == 7:
            current_question['解析'] = line[3:] if line.startswith('解析：') else line
            # 将当前问题添加到数据列表中并重置字典
            data.append(current_question)
            current_question = {}

    # 将数据列表转换为JSON格式
    json_data = json.dumps(data, ensure_ascii=False, indent=4)
    return json_data


with open(transjson_path, 'w', encoding='utf-8') as output_file:
    output_file.write(convert_to_json(filtered_output_path))
